In [1]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
v = CountVectorizer(ngram_range=(1,3))
v.fit(['Thor Hathodwala s looking for a job just like me :)'])
v.vocabulary_

{'thor': 18,
 'hathodwala': 3,
 'looking': 14,
 'for': 0,
 'job': 6,
 'just': 9,
 'like': 12,
 'me': 17,
 'thor hathodwala': 19,
 'hathodwala looking': 4,
 'looking for': 15,
 'for job': 1,
 'job just': 7,
 'just like': 10,
 'like me': 13,
 'thor hathodwala looking': 20,
 'hathodwala looking for': 5,
 'looking for job': 16,
 'for job just': 2,
 'job just like': 8,
 'just like me': 11}

In [6]:
corpus =[
    "thor ate pizza",
    "laila is tall",
    "laila is cooling water",
]

In [7]:
import spacy
nlp = spacy.load("en_core_web_sm")


def preprocessing(text):
    doc = nlp(text)

    filtered_token= []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_token.append(token.lemma_)

    return " ".join(filtered_token)


In [8]:
preprocessing("thor ate pizza")# convert ate to eat the pas verb

'thor eat pizza'

In [9]:
corpus_process = [preprocessing(text) for text in corpus]

In [10]:
corpus_process

['thor eat pizza', 'laila tall', 'laila cool water']

In [11]:
v = CountVectorizer(ngram_range=(1,2))
v.fit(corpus_process)
v.vocabulary_

{'thor': 9,
 'eat': 2,
 'pizza': 7,
 'thor eat': 10,
 'eat pizza': 3,
 'laila': 4,
 'tall': 8,
 'laila tall': 6,
 'cool': 0,
 'water': 11,
 'laila cool': 5,
 'cool water': 1}

In [13]:
vectore = v.transform(["thor ate pizza"])

In [15]:
vectore.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0]], dtype=int64)

In [17]:
import pandas as pd

df = pd.read_json("News_Category_Dataset.json" ,lines=True )
df.head(15)

,short_description,headline,date,link,authors,category
0,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...,2018-05-26,https://www.huffingtonpost.com/entry/texas-ama...,Melissa Jeltsen,CRIME
1,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2...,2018-05-26,https://www.huffingtonpost.com/entry/will-smit...,Andy McDonald,ENTERTAINMENT
2,The actor and his longtime girlfriend Anna Ebe...,Hugh Grant Marries For The First Time At Age 57,2018-05-26,https://www.huffingtonpost.com/entry/hugh-gran...,Ron Dicker,ENTERTAINMENT
3,The actor gives Dems an ass-kicking for not fi...,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,2018-05-26,https://www.huffingtonpost.com/entry/jim-carre...,Ron Dicker,ENTERTAINMENT
4,"The ""Dietland"" actress said using the bags is ...",Julianna Margulies Uses Donald Trump Poop Bags...,2018-05-26,https://www.huffingtonpost.com/entry/julianna-...,Ron Dicker,ENTERTAINMENT
5,"""It is not right to equate horrific incidents ...",Morgan Freeman 'Devastated' That Sexual Harass...,2018-05-26,https://www.huffingtonpost.com/entry/morgan-fr...,Ron Dicker,ENTERTAINMENT
6,"It's catchy, all right.",Donald Trump Is Lovin' New McDonald's Jingle I...,2018-05-26,https://www.huffingtonpost.com/entry/donald-tr...,Ron Dicker,ENTERTAINMENT
7,There's a great mini-series joining this week.,What To Watch On Amazon Prime That’s New This ...,2018-05-26,https://www.huffingtonpost.com/entry/amazon-pr...,Todd Van Luling,ENTERTAINMENT
8,"Myer's kids may be pushing for a new ""Powers"" ...",Mike Myers Reveals He'd 'Like To' Do A Fourth ...,2018-05-26,https://www.huffingtonpost.com/entry/mike-myer...,Andy McDonald,ENTERTAINMENT
9,You're getting a recent Academy Award-winning ...,What To Watch On Hulu That’s New This Week,2018-05-26,https://www.huffingtonpost.com/entry/hulu-what...,Todd Van Luling,ENTERTAINMENT


In [18]:
df.category.value_counts()

category
POLITICS          32739
ENTERTAINMENT     14257
HEALTHY LIVING     6694
QUEER VOICES       4995
BUSINESS           4254
SPORTS             4167
COMEDY             3971
PARENTS            3955
BLACK VOICES       3858
THE WORLDPOST      3664
WOMEN              3490
CRIME              2893
MEDIA              2815
WEIRD NEWS         2670
GREEN              2622
IMPACT             2602
WORLDPOST          2579
RELIGION           2556
STYLE              2254
WORLD NEWS         2177
TRAVEL             2145
TASTE              2096
ARTS               1509
FIFTY              1401
GOOD NEWS          1398
SCIENCE            1381
ARTS & CULTURE     1339
TECH               1231
COLLEGE            1144
LATINO VOICES      1129
EDUCATION          1004
Name: count, dtype: int64